In [1]:
%matplotlib inline
import numpy as np
from pybbn.gaussian.inference import GaussianInference
import matplotlib.pyplot as plt

np.random.seed(37)
plt.style.use('ggplot')

def get_cowell_data():
    n = 10000
    Y = np.random.normal(0, 1, n)
    X = np.random.normal(Y, 1, n)
    Z = np.random.normal(X, 1, n)

    D = np.vstack([Y, X, Z]).T
    return D, ['Y', 'X', 'Z']

def get_mvn():
    X, H = get_cowell_data()

    M = X.mean(axis=0)
    E = np.cov(X.T)
    
    g = GaussianInference(H, M, E)
    return g

g = get_mvn()

In [2]:
g.marginals

[{'name': 'Y', 'mean': -0.0017234068142374496, 'var': 0.9907002440358944},
 {'name': 'X', 'mean': 0.007447599406730594, 'var': 2.0164056677312474},
 {'name': 'Z', 'mean': 0.002459042532737774, 'var': 3.033837891423978}]

In [6]:
from ipywidgets import interact

samples1 = g.sample_marginals(size=10000)
    
@interact(x=(-5, 5, 1))
def f(x=None):
    if x is not None:
        gg = g.do_inference('X', x)
    else:
        gg = g
    
    samples2 = gg.sample_marginals(size=10000)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 3))
    axes = np.ravel(axes)
    
    kind = 'kde'
    for (name, s2), ax in zip(samples2.items(), axes):        
        _ = samples1[name].plot(kind=kind, ax=ax, color='blue', alpha=0.5)
        _ = s2.plot(kind=kind, ax=ax)
        _ = ax.set_title(f'{name}')

interactive(children=(IntSlider(value=0, description='x', max=5, min=-5), Output()), _dom_classes=('widget-int…